In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, train_test_split

In [2]:
from matplotlib.font_manager import FontProperties
import seaborn as sns

In [3]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [4]:
import os
import pandas as pd
import torch

In [5]:
import Hartmannb

The result of custom_function at [0.95, 0.95, 0.95, 0.95, 0.95, 0.95] is 0.00011397265188861638


In [6]:
from botorch.test_functions.base import BaseTestProblem
from botorch.test_functions import Hartmann
import numpy as np 
import torch 
import random 
from botorch.models import SingleTaskGP
from botorch.models.transforms.input import Normalize
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
# from botorch.fit import fit_gpytorch_model 
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf
from botorch.acquisition.objective import ScalarizedPosteriorTransform
from botorch.acquisition import qLogNoisyExpectedImprovement
from botorch.acquisition import qLogExpectedImprovement
import os

In [7]:
from scipy import stats

In [8]:
# 读取 Excel 文件
df = pd.read_csv('data.csv')  # 替换为你的文件名和工作表名

# 显示数据
print(df.head())

            G           F            W          V1          V2         V3  \
0  960.274125   85.293927  6623.437770  214.227575  340.372233  54.603440   
1  959.872112   85.745979  6595.996008  213.992957  332.089900  54.645951   
2  960.379664   86.963096  6622.916667  214.070897  343.915356  54.703336   
3  960.241819   94.040876  6654.799367  214.146170  339.090909  54.797417   
4  955.264501  101.057077  6768.130288  214.118549  369.283862  54.841628   

          C  
0  0.089293  
1  0.090599  
2  0.097921  
3  0.105241  
4  0.119251  


In [9]:
x = df[['G','F', 'W','V1','V2','V3'
        ]]
y = df[['C']]
J = (df['G'] * (1 - df['C']) + df['F']) / df['W']

In [10]:
# 将 y 分为训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

# 打印 y_train 和 y_test 的大小
print(f"Training labels size: {y_train.shape}")
print(f"Test labels size: {y_test.shape}")

Training labels size: (2610, 1)
Test labels size: (290, 1)


In [11]:
# 创建随机森林模型
rf_model = RandomForestRegressor(random_state=42, n_estimators=100)  # 可以根据需要调整参数

# 拟合模型
rf_model.fit(x_train, y_train)

# 在训练集和测试集上进行预测
y_train_pred = rf_model.predict(x_train)
y_test_pred = rf_model.predict(x_test)

# 输出训练集和测试集的 R² 值
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Training R²: {train_r2}")
print(f"Test R²: {test_r2}")

C:\Users\10925\AppData\Local\Temp\ipykernel_1704\600102426.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Training R²: 0.9806131963064145
Test R²: 0.8921716907603638


In [12]:
def predict_function(new_data):
    """给定新的 6 维输入数据，返回 j 的预测值"""
    
    # 将输入数据转换为 NumPy 数组
    new_data_numpy = new_data.cpu().numpy().astype(np.float64)  
    
    # 将 NumPy 数组转换为 pandas DataFrame，并提供列名
    feature_names = ['G','F', 'W','V1','V2','V3']  # 确保与输入数据对应
    new_data_df = pd.DataFrame(new_data_numpy, columns=feature_names)
    
    # 使用随机森林模型进行预测，确保特征顺序正确
    j = rf_model.predict(new_data_df[['G','F', 'W','V1','V2','V3']])  # 使用六个特征进行预测
    
    return torch.tensor(j, dtype=torch.float64)  # 返回 PyTorch 张量，确保为 float64

# 示例输入，确保为6个特征的数组
input_data = np.array([[52, 52, 74, 752, 782, 872]])  # 替换为实际值
input_tensor = torch.tensor(input_data, dtype=torch.float64)

# 调用函数
result = predict_function(input_tensor)

print(result)  # 输出预测结果



tensor([0.0636], dtype=torch.float64)


In [13]:
def generate_random_6d_tensor(n_samples, fixed_values, bounds, device):
    """
    生成一组随机的 6 维张量数据。
    第 2 和第 3 个维度是固定的。
    第 1，4，第 5 和第 6 个维度是随机生成的，具有各自的范围。
    """
    # 检查 bounds 是否为 4 维
    if len(bounds) != 4:
        raise ValueError("Bounds should have 4 elements for the remaining dimensions.")
    
    # 生成随机数据
    random_data = torch.empty((n_samples, 6), device=device)

    # 固定第 2 和第 3 维度的值
    random_data[:, 1] = fixed_values[0]  # 固定值 for 第 2 维
    random_data[:, 2] = fixed_values[1]  # 固定值 for 第 3 维

    # 生成随机范围数据
    while True:
        # 生成随机值
        random_data[:, 0] = torch.FloatTensor(n_samples).uniform_(*bounds[0]).to(device)  # 第1维度 (G)
        random_data[:, 3] = torch.FloatTensor(n_samples).uniform_(*bounds[1]).to(device)  # 第4维度 (F)
        random_data[:, 4] = torch.FloatTensor(n_samples).uniform_(*bounds[2]).to(device)  # 第5维度 (V1)
        random_data[:, 5] = torch.FloatTensor(n_samples).uniform_(*bounds[3]).to(device)  # 第6维度

        # 计算条件
        G_plus_F = random_data[:, 0] + random_data[:, 3]  # G + F
        V1 = random_data[:, 4]  # V1

        # 计算比例并检查条件
        ratio = G_plus_F / (G_plus_F + V1)
        if (ratio > 0.78).all() and (ratio < 0.85).all():
            break  # 如果所有样本都满足条件，则退出循环

    return random_data

# 固定值
fixed_values = (101.057077, 6768.130288)  # 第2和第3维度的固定值

# 随机范围的 bounds
bounds = [(900, 1200),  # 第1维度的范围
          (0, 372),    # 第4维度的范围
          (0, 740),    # 第5维度的范围
          (0, 126)]    # 第6维度的范围

# 选择设备
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# 存储所有生成的样本
all_random_tensor_data = []

# 生成 10 次随机样本，每次 5 个
for _ in range(10):
    random_tensor_data = generate_random_6d_tensor(n_samples=5, fixed_values=fixed_values, bounds=bounds, device=device)
    all_random_tensor_data.append(random_tensor_data)

# 打印所有生成的随机张量数据
for i, tensor in enumerate(all_random_tensor_data):
    print(f"Sample set {i + 1}:\n{tensor}\n")

Sample set 1:
tensor([[1.0759e+03, 1.0106e+02, 6.7681e+03, 1.2918e+02, 3.0213e+02, 7.2350e+01],
        [1.1889e+03, 1.0106e+02, 6.7681e+03, 1.9630e+02, 3.7445e+02, 6.2671e+00],
        [1.0791e+03, 1.0106e+02, 6.7681e+03, 1.4851e+02, 2.5804e+02, 5.2469e+01],
        [9.6116e+02, 1.0106e+02, 6.7681e+03, 3.4610e+02, 3.0421e+02, 1.1273e+02],
        [1.0443e+03, 1.0106e+02, 6.7681e+03, 2.5983e+02, 2.8301e+02, 1.1072e+02]],
       device='cuda:0')

Sample set 2:
tensor([[1.0216e+03, 1.0106e+02, 6.7681e+03, 2.1749e+02, 2.3416e+02, 4.5258e+01],
        [1.0242e+03, 1.0106e+02, 6.7681e+03, 1.7655e+02, 2.3518e+02, 7.0730e+01],
        [9.2427e+02, 1.0106e+02, 6.7681e+03, 9.2844e+01, 1.8860e+02, 3.5846e-01],
        [1.0311e+03, 1.0106e+02, 6.7681e+03, 1.4768e+02, 2.6140e+02, 1.0167e+02],
        [1.0219e+03, 1.0106e+02, 6.7681e+03, 4.3508e+01, 2.0602e+02, 4.7682e+00]],
       device='cuda:0')

Sample set 3:
tensor([[ 978.9510,  101.0571, 6768.1304,  281.9436,  333.0673,  116.8611],
        [ 

In [14]:
def fit_gp_model_and_select_next_point1(train_x, train_obj, device):
    # 确保 train_x 和 train_obj 是 float64 类型
    train_x = train_x.to(device=device, dtype=torch.float64)
    train_obj = train_obj.to(device=device, dtype=torch.float64)

    # 创建和拟合高斯过程模型
    model = SingleTaskGP(
        train_X=train_x,
        train_Y=train_obj,
        input_transform=Normalize(d=train_x.shape[1]),  # 使用输入的维度
        outcome_transform=Standardize(m=1),  # 输出为一维
    ).to(device)

    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)

    # 找到当前最优值
    best_value = train_obj.max()

    # 定义 Log Expected Improvement
    qLogEI = qLogExpectedImprovement(model=model, best_f=best_value)

    # 固定值
    fixed_value_2 =101.057077  # 固定第2维度
    fixed_value_3 = 6768.13028  # 固定第3维度

    # 修改为你指定的上下界，确保第2和第3维度固定
    lower_bounds = [900, fixed_value_2, fixed_value_3, 0, 0, 0]  # 对应 4 维输入
    upper_bounds = [1200, fixed_value_2, fixed_value_3, 372, 740, 126]  # 对应 4 维输入

    # 转换为 tensor，确保数据类型为 float64
    bounds = torch.tensor([lower_bounds, upper_bounds], device=device, dtype=torch.float64)

    while True:
        # 优化获取函数，找到新的采样点
        new_point_analytic, _ = optimize_acqf(
            acq_function=qLogEI,
            bounds=bounds,
            q=1,  # 采样一个点
            num_restarts=20,
            raw_samples=100,
            options={},
        )

        # 将新点转换为张量并提取值
        point = new_point_analytic.squeeze(0)  # 从 (1, 6) 转为 (6,)

        G = point[0].item()  # 第1维度
        F = point[1].item()  # 第4维度
        V1 = point[3].item()  # 第5维度

        # 计算比例
        ratio = (G + F) / (G + F + V1)

        # 检查条件是否满足
        if 0.78 < ratio < 0.85:
            break  # 满足条件，退出循环

    return new_point_analytic.squeeze(0)


In [15]:
results1 = Hartmannb.bayesian_optimization_experiment(
    num_experiments=1,
    n=25,
    obj_fn1=predict_function,
    obj_fn2=None, 
    obj_fn3=predict_function,
    initial_points_task1=all_random_tensor_data,

    fit_task_fn=fit_gp_model_and_select_next_point1,
    device='cuda',
    task_type='single'  # 表示运行单任务模型
)

tensor([[1.0759e+03, 1.0106e+02, 6.7681e+03, 1.2918e+02, 3.0213e+02, 7.2350e+01],
        [1.1889e+03, 1.0106e+02, 6.7681e+03, 1.9630e+02, 3.7445e+02, 6.2671e+00],
        [1.0791e+03, 1.0106e+02, 6.7681e+03, 1.4851e+02, 2.5804e+02, 5.2469e+01],
        [9.6116e+02, 1.0106e+02, 6.7681e+03, 3.4610e+02, 3.0421e+02, 1.1273e+02],
        [1.0443e+03, 1.0106e+02, 6.7681e+03, 2.5983e+02, 2.8301e+02, 1.1072e+02]],
       device='cuda:0')


F:\anaconda\Lib\site-packages\botorch\models\utils\assorted.py:174: InputDataWarning:

Input data is not contained to the unit cube. Please consider min-max scaling the input data.



KeyboardInterrupt: 

In [ ]:
results_list = [results1]
results_names = ['results1']

# 初始化一个空字典来存储展平后的数据
flattened_results = {}

# 遍历所有结果数组
for i, results in enumerate(results_list):
    if isinstance(results, torch.Tensor):
        results_np = results.cpu().numpy()  # 转为 numpy 数组
    else:
        results_np = results  # 如果已经是 numpy 数组，直接使用
    
    # 获取数组的维度
    shape_len = len(results_np.shape)
    
    if shape_len == 3:
        # 3D 数组的形状 (num_experiments, num_samples, num_metrics)
        num_experiments, num_samples, num_metrics = results_np.shape
        # 展平为 2D 数组
        flattened_array = results_np.reshape(num_experiments * num_samples, num_metrics)
    elif shape_len == 2:
        # 2D 数组的形状 (num_experiments, num_metrics)
        flattened_array = results_np
        num_experiments, num_metrics = flattened_array.shape
    else:
        raise ValueError(f"Unexpected number of dimensions: {shape_len}")
    
    # 将每个数组的列命名为 'results1_Metric_1', 'results1_Metric_2', ...
    columns = [f"{results_names[i]}_Metric_{j+1}" for j in range(num_metrics)]
    
    # 将展平后的数组存入字典，方便后面转换为 DataFrame
    flattened_results.update({col: flattened_array[:, j] for j, col in enumerate(columns)})

# 创建 DataFrame
df_results = pd.DataFrame(flattened_results)

# 文件名
csv_filename = '实验数据二'

# 检查 CSV 文件是否已经存在
if os.path.exists(csv_filename):
    # 如果文件存在，先读取现有文件
    df_existing = pd.read_csv(csv_filename)
    # 合并新结果到现有的 DataFrame 中
    df_results = pd.concat([df_existing, df_results], ignore_index=True)

# 保存或追加为 CSV 文件
df_results.to_csv(csv_filename, index=False)

print(f"所有结果已保存到 {csv_filename} 文件中。")

In [ ]:
def load_and_plot_individual_results(csv_files, confidence_level=0.95):
    """
    参数:
    - csv_files: CSV 文件名的列表。
    - confidence_level: 置信水平，默认值为 0.95。
    """
    # 计算置信区间的 Z 值
    z_value = stats.norm.ppf(1 - (1 - confidence_level) / 2)
    
    # 准备线条颜色和标签
    line_colors = ['blue']
    line_styles = ['-']  # 为每一条线添加样式
    labels = ['mokuangshuju']

    # 创建一个包含 2x2 网格的图形
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))

    for idx, csv_filename in enumerate(csv_files):
        # 检查 CSV 文件是否存在
        if not os.path.exists(csv_filename):
            raise FileNotFoundError(f"File {csv_filename} does not exist.")

        # 从 CSV 文件中读取数据
        df_results = pd.read_csv(csv_filename)



        means_bayes1 = df_results.filter(like='results1_Metric_').mean(axis=0)
        standard_errors1 = df_results.filter(like='results1_Metric_').sem(axis=0)
        ci_bayes1 = z_value * standard_errors1

        
        # x轴的索引
        x = np.arange(len(means_bayes1))

        # 找到对应的子图位置
        ax = axs[idx // 2, idx % 2]

        # 使用不同颜色绘制带有误差条的均值图
        ax.errorbar(x, means_bayes1, yerr=ci_bayes1, fmt='o', label='STGP+qlogEI', 
                    color=line_colors[0], linestyle=line_styles[0], ecolor=line_colors[0], capsize=5)
        
        # 设置子图的标题和标签
        ax.set_title(f'Optimal values and confidence intervals ({labels[idx]})')
        ax.set_xlabel('Number of Iterations')
        ax.set_ylabel('The optimal value')

        # 添加图例
        ax.legend()

    # 调整布局
    fig.tight_layout()

    # 保存图像
    plt.savefig('Ackley_Cos_Hartmann_Individual_Plots_Colored_Lines.png', dpi=300)
    # 显示图形
    plt.show()

In [ ]:
# 定义文件名列表
csv_files = [ '实验数据二']

# 调用函数并绘图
load_and_plot_individual_results(csv_files)